In [1]:
!nvidia-smi

Sun Mar 20 05:45:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 11449, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11449 (delta 3), reused 9 (delta 3), pack-reused 11436
Receiving objects: 100% (11449/11449), 11.27 MiB | 15.27 MiB/s, done.
Resolving deltas: 100% (7905/7905), done.


In [ ]:
!pip install -qU wandb
!pip install -qU bbox-utility

In [ ]:
!unzip '/content/drive/MyDrive/input/happy-whale-and-dolphin.zip' -d '/content/data/'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip '/content/drive/MyDrive/Colab Notebooks/whale_share/fold0_labels.zip' -d /content/data # origin
!unzip '/content/drive/MyDrive/input/fold0_labels.zip' -d /content/data

In [ ]:
!rm -rf /content/data/*.txt

In [1]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed

from IPython.display import display, HTML

from matplotlib import animation, rc
rc('animation', html='jshtml')

In [3]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: Currently logged in as: anony-mouse-220166 (use `wandb login --relogin` to force relogin)


To use your W&B account,
Go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. 
Get your W&B access token from here: https://wandb.ai/authorize


In [2]:
FOLD      = 1 # which fold to train
DIM       = 2048
MODEL     = 'yolov5m6'
BATCH     = 4
EPOCHS    = 100

PROJECT   = 'whale' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}' # w&b for yolov5

REMOVE_NOBBOX = True # remove images with no bbox
ROOT_DIR  = '/content/data'
IMAGE_DIR = '/content/images' # directory to save images
LABEL_DIR = '/content/labels' # directory to save labels

In [ ]:
!rm -rf /content/images
!rm -rf /content/labels

In [11]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [12]:
os.makedirs('/content/images/train', exist_ok=True)
os.makedirs('/content/images/val', exist_ok=True)
os.makedirs('/content/labels/train', exist_ok=True)
os.makedirs('/content/labels/val', exist_ok=True)

In [ ]:
len(sorted(glob('/content/data/*.txt')))

5549

In [13]:
from glob import glob
from shutil import copyfile

def make_dataset(mode='train'):
    txt_paths = sorted(glob('/content/data/*.txt'))
    if mode == 'train':
        pgb = txt_paths[:int(len(txt_paths)*0.8)]
    else:
        pgb = txt_paths[int(len(txt_paths)*0.8):]
    for i in pgb:
        file_name = i.split('/')[-1].split('.')[0].split('_')[0]
        copyfile('/content/data/test_images/'+file_name+'.jpg', f'/content/images/{mode}/' + file_name + '.jpg')
        f = open(i, 'r')
        k = open(f'/content/labels/{mode}/'+file_name+'.txt', 'w')
        for j in f.readlines():
            k.write(j)

        # img = plt.imread('/content/data/test_images/' + file_name+'.jpg')
        # plt.imshow(img)
        # plt.show()
        f.close()
        k.close()
        # break

In [14]:
make_dataset(mode='train')
make_dataset(mode='val')

In [15]:
import yaml

cwd = '/content/'

data = dict(
    path  = '/content',
    train =  '/content/images/train' ,
    val   =  '/content/images/val',
    nc    = 1,
    names = ['fin'],
    )

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- fin
nc: 1
path: /content
train: /content/images/train
val: /content/images/val



In [1]:
%%writefile /content/hyp.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.1 # 0.30  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.10  # image scale (+/- gain)
shear: 0.0 # 2.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.0 # 0.2  # image mosaic (probability)
mixup: 0.0 # 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

Overwriting /content/hyp.yaml


In [3]:
%cd /content
!rm -r /kaggle/working/yolov5
# !git clone https://github.com/ultralytics/yolov5 # clone
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

YOLOv5 🚀 v6.1-40-gb0ba101 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 102.4/166.8 GB disk)


In [ ]:
!python train.py --img {DIM} \
--batch {BATCH} \
--epochs {EPOCHS} \
--data /content/gbr.yaml \
--hyp /content/hyp.yaml \
--weights {MODEL}.pt \
--project {PROJECT} --name {NAME} \
--exist-ok

wandb: Currently logged in as: anony-mouse-220166 (use `wandb login --relogin` to force relogin)
train: weights=yolov5m6.pt, cfg=, data=/content/gbr.yaml, hyp=/content/hyp.yaml, epochs=100, batch_size=4, imgsz=2048, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=whale, name=yolov5m6-dim2048-fold1, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 52 (delta 34), reused 33 (delta 31), pack-reused 8
Unpacking objects: 100% (52/52), done.
From https://github.com/ultralytics/yolov5
   b0ba101..529fbc1  master              -> origin

In [ ]:
3
